In [1]:
# INTEL CORPORATION CONFIDENTIAL AND PROPRIETARY
# 
# Copyright © 2019-2021 Intel Corporation.
# 
# This software and the related documents are Intel copyrighted
# materials, and your use of them is governed by the express 
# license under which they were provided to you (License). Unless
# the License provides otherwise, you may not use, modify, copy, 
# publish, distribute, disclose or transmit  this software or the
# related documents without Intel's prior written permission.
# 
# This software and the related documents are provided as is, with
# no express or implied warranties, other than those that are 
# expressly stated in the License.

In [2]:
import os
import nxsdk.api.n2a as nx
import numpy as np
import matplotlib.pyplot as plt
from nxsdk.graph.monitor.probes import *

net = nx.NxNet()

# Read Lakemont Spike Count
This tutorial illustrates the use of spike probes to capture the spikes count from lakemont via SNIP. Normally spike probe involves frequent interaction between host and Lakmont on chip, which could slow down the application while host keeps on reading out spike information from Lakemont then sending it back to spike probe. In situation when we want spikes count to be returned as soon as possible, we can use SNIP to read out spikes count register directly from Lakmont through a channel, rather than relying on the spike probe returned from host. 
## Create a network
A simple network is created here, with one spike generator sending one spike at every time step to one compartment.

In [3]:
# create a compartment prototype
cxp = nx.CompartmentPrototype(biasMant=1000,
                              biasExp=6,
                              vThMant=10000,                              
                              compartmentVoltageDecay=0,
                              functionalState=nx.COMPARTMENT_FUNCTIONAL_STATE.IDLE)
# Create a compartment from prototype cxp
cx = net.createCompartment(cxp)

# create a spike generator spikeGen that sends spike at each time step
numPorts = 1
spikeGen = net.createSpikeGenProcess(numPorts)
spikeTimes0 = list(range(100))
spikeGen.addSpikes([0], [spikeTimes0])

# connect spikeGen with compartment cx
connProto = nx.ConnectionPrototype(weight=200, signMode=nx.SYNAPSE_SIGN_MODE.MIXED)
spikeGen.connect(cx, prototype=connProto)

## Set up spike probe
The spike probe sProbe is setup for compartment cx. With a setup spike probe, an output axon is created internally from cx to Lakmont spike counter register located at 0x20, and a SNIP is able to read out the spike count information directly from that register address. Probe counter register addresses are assigned incrementally, so if we create one more probe, its address at Lakemont would be 0x21.

In this tutorial, the spike probe is not used to display, as this could slow down application. Here we set tStart = 10000000 (any large value) so that the probe infrastrucutre on host won't start to read until very late at tStart. Basically, we disable spike probe reading from the beginning. 

In [4]:
customSpikeProbeCond = SpikeProbeCondition(tStart=10000000)
sProbe = cx.probe(nx.ProbeParameter.SPIKE, customSpikeProbeCond)

## Compile network

In [5]:
compiler = nx.N2Compiler()
board = compiler.compile(net)

## Configure SNIP and channels 
Since we basically disable spike probe reading at the beginning, there is no spike count information in spike probe itself. To get spike count information, we can setup a SNIP to read out spike count information directly from Lakemont register.

In [6]:
# Define directory where SNIP C-code is located
includeDir = os.getcwd()

# Create SNIP, define which code to execute and in which phase of the NxRuntime execution cycle
runMgmtProcess = board.createProcess("runMgmt",
                                     includeDir=includeDir,
                                     cFilePath = includeDir + "/runmgmt.c",
                                     funcName = "run_mgmt",
                                     guardName = "do_run_mgmt",
                                     phase = "mgmt")
                          
# Create a channel named spikeCntr to get the spikes count information from Lakemont
spikeCntrChannel = board.createChannel(b'nxspkcntr', "int", 100)

# Connecting spikeCntr from runMgmtProcess to SuperHost which is receiving spike count in the channel
spikeCntrChannel.connect(runMgmtProcess, None)

## Execute network
When the network runs, the SNIP will read the spike count value from Lakemont, then write it to spikeCntrChannel, which will be printed out on screen.  

In [7]:
board.run(100, aSync=True)

spikeCntr = []
spikeCntr.append(spikeCntrChannel.read(100))
print(spikeCntr)

board.finishRun()
board.disconnect()

INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 134.134.68.81:18861
[(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11)]


.. code-links:: python clear